Pull in firstpage of reports and find out how many pages there are in total

In [1]:
import requests
from bs4 import BeautifulSoup
URL = 'https://www.gov.uk/service-standard-reports'
assessments_pg1 = requests.get(URL)
assessments_pg1_content = BeautifulSoup(assessments_pg1.content, 'html.parser')

import re
pagination_label = assessments_pg1_content.find('span',class_ = 'gem-c-pagination__link-label').text.strip()
# do I need to add something if there is just one page? probably never happen?
final_page = re.search('.* of (.*)', pagination_label, re.IGNORECASE).group(1)


In [2]:
def get_assessment_date(content,service_detail):
    assessment_date = ''
    table_results = content.find_all('div',class_ = 'app-c-important-metadata')
    for row in table_results:
        row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
        if(re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_date = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(2).strip()
            service_detail['attributes']['assessment_date'] = assessment_date
        if(assessment_date == '' and re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_date = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(2).strip()
            service_detail['attributes']['assessment_date'] = assessment_date
        if(assessment_date == '' and re.search('(Assessment date:)\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_date = re.search('(Assessment date:)\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(2).strip()
            service_detail['attributes']['assessment_date'] = assessment_date
    table_results = content.find_all('tr')
    if assessment_date == '':
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^Assessment date:.*', row_text, re.IGNORECASE):
                assessment_date = re.search('^(Assessment date:)(.*)', row_text, re.IGNORECASE).group(2).strip()
                service_detail['attributes']['assessment_date'] = assessment_date
    return(service_detail)

In [3]:
def get_assessment_result(content,service_detail):
    assessment_result = ''
    table_results = content.find_all('div',class_ = 'app-c-important-metadata')
    for row in table_results:
        row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
        if(re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_result = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(3).strip()
            service_detail['attributes']['assessment_result'] = assessment_result
        if(assessment_result == '' and re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_result = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(3).strip()
            service_detail['attributes']['assessment_result'] = assessment_result
    table_results = content.find_all('tr')
    if assessment_result == '':
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^(assessment ){0,1}Result.*', row_text, re.IGNORECASE):
                assessment_result = re.search('^(assessment ){0,1}(Result)(:){0,1}(.*)', row_text, re.IGNORECASE).group(4).strip()
                service_detail['attributes']['assessment_result'] = assessment_result
    if assessment_result == '':
        p_results = content.find_all('p')
        for p in p_results:
            p_text = p.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^Result.*', p_text, re.IGNORECASE):
                assessment_result = re.search('^(assessment ){0,1}(Result)(:){0,1}(.*)', p_text, re.IGNORECASE).group(4).strip()
                service_detail['attributes']['assessment_result'] = assessment_result
    return(service_detail)

In [4]:
def get_assessment_service_provider(content,service_detail):
    service_provider = ''
    if service_provider == '':
        table_results = content.find_all('tr')
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^Service provider.*', row_text, re.IGNORECASE):
                service_provider = re.search('^(Service provider)(:){0,1}(.*)', row_text, re.IGNORECASE).group(3).strip()
                service_detail['attributes']['service_provider'] = service_provider
    if service_provider == '':
        p_results = content.find_all('p')
        for p in p_results:
            p_text = p.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^Service provider.*', p_text, re.IGNORECASE):            
                service_provider = re.search('^(Service provider)(:){0,1}(.*)', p_text, re.IGNORECASE).group(3).strip()
            if re.search('^Department / Agency.*', p_text, re.IGNORECASE):
                service_provider = re.search('^(Department / Agency)(:){0,1}(.*)', p_text, re.IGNORECASE).group(3).strip()
            service_detail['attributes']['service_provider'] = service_provider
   

In [5]:
def get_assessment_stage(content,service_detail):
    assessment_stage = ''
    table_results = content.find_all('div',class_ = 'app-c-important-metadata')
    for row in table_results:
        row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
        if(re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_stage = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Result:\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(4).strip()
            service_detail['attributes']['assessment_stage'] = assessment_stage
        if(assessment_stage == '' and re.search('(Assessment date:)\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_stage = re.search('(Assessment date:)\s{0,}(.*)\s{0,}Stage:\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(3).strip()
            service_detail['attributes']['assessment_stage'] = assessment_stage
        if(assessment_stage == '' and re.search('(Stage:)\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE)):
            assessment_stage = re.search('(Stage:)\s{0,}(.*)\s{0,}', row_text, re.IGNORECASE).group(2).strip()
            service_detail['attributes']['assessment_stage'] = assessment_stage
    table_results = content.find_all('tr')
    if assessment_stage == '':
        for row in table_results:
            row_text = row.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^(Assessment ){0,1}(Stage)(:){0,1}(.*)', row_text, re.IGNORECASE):
                assessment_stage = re.search('^(Assessment ){0,1}(Stage)(:){0,1}(.*)', row_text, re.IGNORECASE).group(4).strip()
                service_detail['attributes']['assessment_stage'] = assessment_stage
    if assessment_stage == '':
        p_results = content.find_all('p')
        for p in p_results:
            p_text = p.text.strip().replace('\n', ' ').replace('\r', '')
            if re.search('^(Assessment ){0,1}(Stage)(:){0,1}(.*)', p_text, re.IGNORECASE):
                assessment_stage = re.search('^(Assessment ){0,1}(Stage)(:){0,1}(.*)', p_text, re.IGNORECASE).group(4).strip()
            if re.search('^Moving to:.*', p_text, re.IGNORECASE):
                assessment_stage = re.search('^(Moving to)(:){0,1}(.*)', p_text, re.IGNORECASE).group(3).strip()
            service_detail['attributes']['assessment_stage'] = assessment_stage
    return(service_detail)

In [6]:
def get_service_details(content,service_detail):
    get_assessment_stage(content,service_detail)
    get_assessment_date(content,service_detail)
    get_assessment_result(content,service_detail)
    get_assessment_service_provider(content,service_detail)
    return(service_detail)

Get all the individual links for all pages plus some summary information that's provided for some assessments

In [7]:
URL = 'https://www.gov.uk/service-standard-reports?page='
i = 1
service_details = []
while i <= int(final_page):
  page = requests.get(URL+str(i))
  content = BeautifulSoup(page.content, 'html.parser')
  results = content.find_all('li', class_='gem-c-document-list__item')
  for result in results: 
          attribute_array = {"service":[],'attributes':{"url":[],"assessment_result":[''],"assessment_stage":[''],"assessment_date":[''],'service_provider':[''],'note':['']}}
          title_html = result.find('a', class_='gem-c-document-list__item-title gem-c-document-list__item-link')
          title_text = title_html.text.strip()
          attribute_array['service'] = title_text
          url = result.find('a')['href']
          attribute_array['attributes']['url'] = url
          attributes = result.find_all('li', class_='gem-c-document-list__attribute')
          if attribute_array['attributes']['url'] != "/service-standard-reports/nhs-111":
            service_details.append(attribute_array)
  i += 1

In [8]:
from datetime import datetime
first_service_results = []
for service_detail in service_details:
    point=''
    result = ''
    collect_next_cell = ''
    service_result = {"service_details":[],'results':[],'html_contet':[]}
    extension = service_detail['attributes']['url']
    URL = 'https://www.gov.uk' + extension
    page = requests.get(URL)
    content = BeautifulSoup(page.content, 'html.parser')
    service_result['html_content'] = content 
    results = content.find_all('p')
    service_attributes = content.find_all('dd')
# Get results from newest standard format pages
# these are reports with no table and the decisions within the text     
    service_result['service_details'] = service_detail
    for p in results:
        p_text = p.text.strip()
        if re.search('^The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE):
            result = re.search('The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE).group(2)
            point = re.search('The (team|service) (.*) point (.*) of the Standard', p_text, re.IGNORECASE).group(3)        
            service_result['results'].append({'point':point,'result':result})
        if re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE):
            result = re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE).group(1)
            point = re.search('(met|not met) point (.*) of the Standard', p_text, re.IGNORECASE).group(2)        
            service_result['results'].append({'point':point,'result':result})
    get_service_details(content,service_detail)
    first_service_results.append(service_result)

In [9]:
#Get results from older tables
# # where the links in the table are helpfully numbered
for service in first_service_results:
    if service["results"] == []:
        service_result = {"service_details":[],'results':[],'html_contet':[]}
        for service_detail in service_details:
            if service_detail['service'] == service['service_details']['service']:  
                content = service['html_content']
                service_result['service_details'] = service_detail
                table_results = content.find_all('tr')
                service_result['results'] = []
                for row in table_results:
                    try: href = row.find('a')['href']
                    except TypeError:
                        href = ''
                    if re.search('service-manual.*', str(href), re.IGNORECASE):
                        for cell in row:
                            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                            except:
                                cell_text = ''
                            result = ''
                            if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                                 point = cell_text
                            if re.search('.*met$', cell_text, re.IGNORECASE):
                                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                                    result = 'did not meet'
                                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                                    result = 'met'
                            if point != '' and result != '' and re.search('.*met|meet$', result, re.IGNORECASE): 
                                service_result['results'].append({'point':point,'result':result})
                                point = ''
                service['results'] = service_result['results']
                get_service_details(content,service_detail)
                del service_result
                del content



In [10]:
points_regex ='Understanding user needs|Improving the service based on user research and usability testing|Having a sustainable, multidisciplinary team in place|Building using agile, iterative and user-centred methods|Iterating and improving the service on a frequent basis|Evaluating tools, systems, and ways of procuring them|Managing data, security level, legal responsibilities, privacy issues and risks|Making code available as open source|Using open standards and common government platforms|Testing the end-to-end service, and browser and device testing|Planning for the service being taken temporarily offline|Creating a simple and intuitive service|Ensuring consistency with the design and style of GOV.UK|Encouraging digital take-up|Using analytics tools to collect and act on performance data|Defining KPIs and establishing performance benchmarks|Reporting performance data on the Performance Platform|Testing the service with the minister responsible for it'

In [11]:
#Get results from older tables
# # where the links are broken
for service in first_service_results:
    if service["results"] == []:
        service_result = {"service_details":[],'results':[],'html_contet':[]}
        for service_detail in service_details:
            if service_detail['service'] == service['service_details']['service']:  
                content = service['html_content']
                service_result['service_details'] = service_detail
                table_results = content.find_all('tr')
                service_result['results'] = []
                for row in table_results:
                    if re.search(points_regex, str(row), re.IGNORECASE):
                        for cell in row:
                            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                            except:
                                cell_text = ''
                            result = ''
                            if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                                 point = cell_text
                            if re.search('.*met$', cell_text, re.IGNORECASE):
                                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                                    result = 'did not meet'
                                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                                    result = 'met'
                            if point != '' and result != '' and re.search('.*met|meet$', result, re.IGNORECASE): 
                                service_result['results'].append({'point':point,'result':result})
                                point = ''
                get_service_details(content,service_detail)
                service['results'] = service_result['results']
                del service_result
                del content

In [12]:
#Get results from older tables
# # where the table is just service stanard points as numbers and no link
for service in first_service_results:
    if service["results"] == []: 
        service_result = {"service_details":[],'results':[],'html_contet':[]}
        for service_detail in service_details:
            if service_detail['service'] == service['service_details']['service']:  
                content = service['html_content']
                table_results = content.find_all('tr')
                service_result['results'] = []
                for row in table_results:
                     if re.search('.*Yes.*|.*No.*', str(row), re.IGNORECASE):
                        for cell in row:
                            try: cell_text = cell.text.strip().replace('\n', ' ').replace('\r', '')
                            except:
                                cell_text = ''
                            result = ''
                            if re.search('\d{1,2}', cell_text, re.IGNORECASE):
                                 point = cell_text
                            if re.search('No|Yes', cell_text, re.IGNORECASE):
                                if re.search('No|Fail|not met', cell_text, re.IGNORECASE):
                                    result = 'did not meet'
                                elif re.search('Yes|Pass|Met', cell_text, re.IGNORECASE):
                                    result = 'met'
                            if point != '' and result != '' and re.search('.*met|meet$', result, re.IGNORECASE):
                                service_result['results'].append({'point':point,'result':result})
                                point=''
                service['results'] = service_result['results']
                get_service_details(content,service_detail)
                del service_result
                del content
                            

In [76]:
import pandas as pd
service_result_df = pd.DataFrame()
for service_result in first_service_results:
    service_name = service_result['service_details']['service']
    service_stage = service_result['service_details']['attributes']['assessment_stage']
    assessment_date = service_result['service_details']['attributes']['assessment_date']
    assessment_result = service_result['service_details']['attributes']['assessment_result']
    service_provider = service_result['service_details']['attributes']['service_provider']
    results = service_result['results']
    if len(results)>0:
        service_result_df_raw = pd.DataFrame.from_dict(results).drop_duplicates(subset=["point"], keep='last').set_index('point')
        service_result_df_wide = service_result_df_raw.T
        service_result_df_wide['service_name'] = service_name
        service_result_df_wide['service_stage'] = service_stage
        service_result_df_wide['service_provider'] = service_provider
        service_result_df_wide['assessment_result'] = assessment_result
        service_result_df_wide['assessment_date'] = assessment_date
        service_result_df_wide.set_index('service_name')
        service_result_df = service_result_df.append(service_result_df_wide)
        service_result_df.sort_values(by=['assessment_date','service_name'])
service_result_df.drop_duplicates()
cols = service_result_df.columns.tolist()

for index, row in service_result_df.iterrows():
    try:
        row['assessment_date'] = date.strptime(row['assessment_date'], "%d %B %Y").date()
    except (ValueError, RuntimeError, TypeError, NameError):
        row['assessment_date'] = date.strptime(row['assessment_date'], "%d/%m/%Y").date()



In [77]:
cols = [int(ele) if ele.isdigit() else ele for ele in cols] 
# comparator function for sort 
def mixs(num): 
    try: 
        ele = int(num) 
        return (0, ele, '') 
    except ValueError: 
        return (1, num, '') 

cols.sort(key = mixs) 
cols = cols[-5:] + cols[:-5]
cols = [str(i) for i in cols]
service_result_df = service_result_df[cols]
service_result_df = service_result_df.set_index('service_name')
service_result_df = service_result_df.sort_values(by =['assessment_date'])
service_result_df.to_csv('service_result_df.csv')
